### Combination of Test 6 % 7
AI agent with memory

In [ ]:
import os
import sys
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.chat_message_histories.upstash_redis import (UpstashRedisChatMessageHistory)
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_8.ipynb')
git_add(file_path)
git_commit('Updated test 8')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

True

Login to https://upstash.com/?utm_source=Leon_LangchainPython
to save the chat history

In [ ]:
! pip install upstash_redis

In [ ]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/FreeSurfer")  # Load the document from the web
docs = loader.load()  # Load the document

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
split_docs = splitter.split(docs)

embedding = OpenAIEmbeddings()  # Generate embeddings using OpenAI
vector_store = FAISS.from_documents(split_docs, embedding=embedding)  # Store documents in FAISS index - Facebook AI Similarity Search


history = UpstashRedisChatMessageHistory(
    url = os.getenv("UPSTASH_REDIS_REST_URL"),
    token = os.getenv("UPSTASH_REDIS_REST_TOKEN"),
    session_id = "chat_1",
    ttl = 0 #conversation will never expirt. This is te time limit in seconds
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.5
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly AI assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True,
    chat_memory=history
)

chain = LLMChain(
    llm = model,
    prompt = prompt,
    memory = memory,
    verbose = False
)

if __name__ == "__main__":

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = chain.run(input=user_input)
        print("AI:", response)



AI: Hello again! How can I help you today?
AI: That's okay! If you think of any questions or need assistance in the future, feel free to reach out. Have a great day!
AI: I am an AI assistant here to help and assist you with any questions or tasks you may have. Feel free to ask me anything!
